# ノードの再起動

---

OpenHPC環境の停止したノードを再起動します。

## VCP の初期化
次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

## 再起動する UnitGroup 名の指定

In [ ]:
ugroup_name = 'TFclusterGPU'

### 操作可能な UnitGroup 名の表示（必要に応じて実行してください）

他の UnitGroup を再起動したい場合は、次のセルを実行して UnitGroup 名を参照し、上のセルの ugroup_name を変更して実行してください。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

## ノードの再起動

### UnitGroup の変数の読み込み

UnitGroup の変数を group_vars ファイルから読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

### ノードの状態確認

現在のノードの状態を確認します。

In [ ]:
ug = vcp.get_ugroup(ugroup_name)
ug.df_nodes()

### マスターノードの再起動

マスターノードを再起動します。

> マスターノードを停止していない場合は、この節をスキップしてください。

In [ ]:
unit_master = ug.get_unit('master')
unit_master.power_on_nodes()

マスターノードの状態を確認します。

In [ ]:
# ★★★ 再起動チェック自動化のためオリジナルを改造 ★★★

rcount = 10             # retry count
while True:
    # マスターノードの状態取得
    unit_master = ug.get_unit('master')
    if any([node.state != 'RUNNING' for node in unit_master.find_nodes()]):
        !sleep 10
        rcount -= 1
        if rcount == 0:
            unit_master.df_nodes()
            raise RuntimeError('Error: cannot restart master node')
    else:
        break

NFSサーバが起動していることを確認します。

In [ ]:
# ★★★ 自動リカバリのためオリジナルを改造 ★★★

rcount = 3             # retry count
while True:
    # NFS サーバの稼働確認
    ustate = !ansible {ugroup_name}_master -b -a 'systemctl status nfs-server'
    for x in range(len(ustate)-1):
        nfsstate = ustate[x]
        if nfsstate.split()[0] == 'Active:':
            break

    print('NFS server: ' + nfsstate)
    
    # 稼働していない場合再起動
    if nfsstate.split()[1] == 'active':
        break
    else:
        !ansible {ugroup_name}_master -b -a 'systemctl restart nfs-server'
        !sleep 5
        rcount -= 1
        if rcount == 0:
            raise RuntimeError('Error: cannot restart nfs server')

### 計算ノードの再起動

計算ノードを再起動します。

In [ ]:
unit_compute = ug.get_unit('compute')
unit_compute.power_on_nodes(num_power_on_nodes=gvars['compute_nodes'])

マスターノードの状態を確認します。

In [ ]:
# ★★★ 再起動チェック自動化のためオリジナルを改造 ★★★

#    unit_compute = ug.get_unit('compute')
rcount = 3
while True:
    if any([node.state != 'RUNNING' for node in unit_compute.find_nodes()]):
        for node in unit_compute.find_nodes():
            if node.state != 'RUNNING':
                unit_compute.power_on_nodes(node_no=node.no)
        !sleep 10
        rcount -= 1
        if rcount == 0:
            unit_compute.df_nodes()
            raise RuntimeError('Error: cannot restart compute node')
    else:
        break

### Slurmの状態を確認する

Slurmクラスタのノードの状態を確認します。

In [ ]:
# ★★★ 自動リカバリのためオリジナルを改造 ★★★

count = 10
while True:
    bs_state = !ansible {ugroup_name}_master -a sinfo
    bs_state = bs_state[len(bs_state)-1]
    print(bs_state)
    
    if bs_state.split()[4] == 'idle':
        break
    elif bs_state.split()[1] == 'Unable':
        !ansible {ugroup_name}_master -b -a 'systemctl restart slurmctld'
        !ansible {ugroup_name}_compute -b -a 'systemctl restart slurmd'
        !sleep 5
    else:
        for x in gvars['compute_etc_hosts'].values():
            !ansible {ugroup_name}_master -b -a 'scontrol update nodename={x} state=resume' || true
        !sleep 5
        
    count -= 1
    if count == 0:
        raise RuntimeError('Error: Slurm is down')

`srun`コマンドで、`hostname` コマンドを計算ノードで実行させてみます。

In [ ]:
!ansible {ugroup_name}_master -a 'srun -l -N {gvars["compute_nodes"]} hostname'

以下のような実行結果が表示されます（ホストの表示順序は入れ替わることがあります）。

```
0: c1

```